# 🎥 AI Attendance System - Local Camera Demo

**Real-time Camera Processing with Employee Folder Auto-Detection**

Pipeline V1: SCRFD Detection + ArcFace Recognition + SQLite Database

---

## 🌟 Features

✅ **Zero-Clone Setup**: Direct pip install, no git operations needed  
✅ **Smart Employee Detection**: Auto-scan employee folders with face validation  
✅ **Real-time Dashboard**: Live processing with performance overlay  
✅ **Instant Notifications**: Local popup notifications for attendance events  
✅ **Persistent Storage**: Local database with backup options  

---


## Cell 1: ⚡ Lightning Setup (No Git Required)

**Direct pip install from PyPI/GitHub with automatic dependency resolution**


In [5]:
# Cell 1: ⚡ Lightning Setup - IMPROVED VERSION
# AI Attendance System - Local Camera Demo
# Lightning Setup with Better Error Handling

import sys
import subprocess
import time
import os
from pathlib import Path
import platform
import importlib
import warnings
warnings.filterwarnings('ignore')

print("⚡ AI ATTENDANCE SYSTEM - LOCAL CAMERA DEMO")
print("=" * 50)

# Environment Detection
print("🔍 ENVIRONMENT DETECTION:")
print(f"├─ Platform: {platform.system()} {platform.release()}")
print(f"├─ Python: {sys.version.split()[0]}")
print(f"├─ Working Directory: {os.getcwd()}")
print(f"└─ Architecture: {platform.machine()}")

# Function to install and verify packages
def install_and_verify_package(package_name, import_name=None, timeout=120):
    """Install package and verify it can be imported"""
    if import_name is None:
        import_name = package_name.replace('-', '_')
    
    try:
        # Try to import first
        importlib.import_module(import_name)
        print(f"├─ ✅ {package_name} (already installed)")
        return True
    except ImportError:
        pass
    
    try:
        # Install package
        print(f"├─ 🔄 Installing {package_name}...")
        result = subprocess.run([
            sys.executable, '-m', 'pip', 'install', package_name, '-q', '--upgrade'
        ], timeout=timeout, capture_output=True, text=True)
        
        if result.returncode == 0:
            # Verify import after installation
            try:
                importlib.import_module(import_name)
                print(f"├─ ✅ {package_name} (installed successfully)")
                return True
            except ImportError:
                print(f"├─ ⚠️ {package_name} (installed but import failed)")
                return False
        else:
            print(f"├─ ❌ {package_name} (installation failed)")
            return False
            
    except subprocess.TimeoutExpired:
        print(f"├─ ⏰ {package_name} (timeout - may still be installing)")
        return False
    except Exception as e:
        print(f"├─ ❌ {package_name} (error: {str(e)[:50]})")
        return False

# GPU Detection
print("\n🚀 GPU DETECTION:")
try:
    import torch
    gpu_available = torch.cuda.is_available()
    if gpu_available:
        print(f"├─ Status: ✅ CUDA Available")
        print(f"├─ Device: {torch.cuda.get_device_name(0)}")
        print(f"└─ Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
    else:
        print(f"├─ Status: ⚠️ CPU Only")
        print(f"└─ Note: Install CUDA for better performance")
except ImportError:
    print(f"├─ Status: 🔄 PyTorch not yet installed")
    gpu_available = False

# Install packages with verification
print("\n📦 INSTALLING DEPENDENCIES:")

# Core packages for local demo - FIXED ORDER
packages_to_install = [
    # Basic packages first
    ('numpy', 'numpy'),
    ('pillow', 'PIL'),
    ('matplotlib', 'matplotlib'),
    ('pandas', 'pandas'),
    ('tqdm', 'tqdm'),
    ('psutil', 'psutil'),
    ('ipywidgets', 'ipywidgets'),
    ('plotly', 'plotly'),
    
    # Computer vision packages
    ('opencv-python', 'cv2'),
    ('scikit-learn', 'sklearn'),
    
    # AI/ML packages
    ('torch', 'torch'),
    ('torchvision', 'torchvision'),
    ('insightface', 'insightface'),
]

successful_installations = 0
failed_packages = []

for package_name, import_name in packages_to_install:
    if install_and_verify_package(package_name, import_name):
        successful_installations += 1
    else:
        failed_packages.append(package_name)

print(f"\n📊 INSTALLATION SUMMARY:")
print(f"├─ Successful: {successful_installations}/{len(packages_to_install)}")
print(f"├─ Failed: {len(failed_packages)}")
if failed_packages:
    print(f"└─ Failed packages: {', '.join(failed_packages)}")

# Create directory structure
base_dir = Path.cwd()
directories = {
    'employees': base_dir / 'employees',
    'data': base_dir / 'data', 
    'snapshots': base_dir / 'snapshots',
    'exports': base_dir / 'exports',
    'logs': base_dir / 'logs'
}

print(f"\n📂 DIRECTORY STRUCTURE:")
for name, path in directories.items():
    path.mkdir(exist_ok=True)
    print(f"├─ {name.title()}: {path}")

# Create sample employee folder structure
sample_employees_dir = directories['employees'] / "_sample_structure"
sample_employees_dir.mkdir(exist_ok=True)

readme_content = """# Employee Registration Setup

## Folder Structure:
- employees/John_Doe/photo1.jpg, photo2.jpg, photo3.jpg
- employees/Jane_Smith/image1.png, image2.png
- employees/Michael_Johnson/face1.jpg, face2.jpg

## Requirements:
- 1-5 clear face photos per employee
- Good lighting and frontal face view
- Supported formats: .jpg, .jpeg, .png
- Minimum resolution: 200x200 pixels

## Naming Convention:
- Folder name = Employee name with underscores
- Examples: John_Doe, Jane_Smith, Michael_Johnson
"""

with open(sample_employees_dir / "README.md", "w") as f:
    f.write(readme_content)

print(f"\n🎉 SETUP COMPLETED!")
print(f"├─ Installation success rate: {successful_installations/len(packages_to_install)*100:.1f}%")

if len(failed_packages) == 0:
    print(f"├─ Status: ✅ All packages installed successfully")
    print(f"└─ Ready for AI processing!")
else:
    print(f"├─ Status: ⚠️ Some packages failed - system may still work")
    print(f"└─ Manual install failed packages: pip install {' '.join(failed_packages)}")

print("\n💡 NEXT STEPS:")
print("1. Add employee photos to employees/ folder")
print("2. Run Cell 2 to initialize AI system")
print("3. Restart kernel if you encounter import errors")
print("=" * 50)

⚡ AI ATTENDANCE SYSTEM - LOCAL CAMERA DEMO
🔍 ENVIRONMENT DETECTION:
├─ Platform: Windows 11
├─ Python: 3.12.8
├─ Working Directory: k:\Workspace\BHK Research\auto-face-attendance\notebooks
└─ Architecture: AMD64

🚀 GPU DETECTION:
├─ Status: ⚠️ CPU Only
└─ Note: Install CUDA for better performance

📦 INSTALLING DEPENDENCIES:
├─ ✅ numpy (already installed)
├─ ✅ pillow (already installed)
├─ ✅ matplotlib (already installed)
├─ ✅ pandas (already installed)
├─ ✅ tqdm (already installed)
├─ ✅ psutil (already installed)
├─ ✅ ipywidgets (already installed)
├─ ✅ plotly (already installed)
├─ ✅ opencv-python (already installed)
├─ ✅ scikit-learn (already installed)
├─ ✅ torch (already installed)
├─ ✅ torchvision (already installed)
├─ ✅ insightface (already installed)

📊 INSTALLATION SUMMARY:
├─ Successful: 13/13
├─ Failed: 0

📂 DIRECTORY STRUCTURE:
├─ Employees: k:\Workspace\BHK Research\auto-face-attendance\notebooks\employees
├─ Data: k:\Workspace\BHK Research\auto-face-attendance\notebooks\d

## Cell 2.5: 🔍 Dependency Verification

**Quick check to ensure all packages are properly installed and importable**

> **💡 Important:** If you see import errors below, follow these steps:
> 1. **Restart the kernel**: Kernel → Restart & Clear Output
> 2. **Re-run setup cell**: Go back and run Cell 2 again
> 3. **Wait for completion**: Ensure all installations finish
> 4. **Run this cell again** to verify everything works


## Cell 2: 🤖 AI System & Database Setup

**Initialize AI models and local database for employee management**


In [6]:
# Cell 2: 🤖 AI System & Database Setup - IMPROVED VERSION
# AI System & Database Setup with Better Error Handling

import sys
import importlib
import warnings
warnings.filterwarnings('ignore')

print("🤖 AI SYSTEM & DATABASE SETUP")
print("=" * 35)

# Function to safely import modules
def safe_import(module_name, package_name=None):
    """Safely import module with fallback installation"""
    try:
        return importlib.import_module(module_name)
    except ImportError as e:
        if package_name:
            print(f"⚠️ {module_name} not found, trying to install {package_name}...")
            try:
                import subprocess
                subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name, '-q'])
                return importlib.import_module(module_name)
            except:
                print(f"❌ Failed to install {package_name}")
                return None
        else:
            print(f"❌ Failed to import {module_name}: {e}")
            return None

# Import required modules with error handling
print("📦 LOADING REQUIRED MODULES:")

# Core imports
cv2 = safe_import('cv2', 'opencv-python')
np = safe_import('numpy')
pd = safe_import('pandas')
plt = safe_import('matplotlib.pyplot')
json = safe_import('json')
sqlite3 = safe_import('sqlite3')
time = safe_import('time')
logging = safe_import('logging')

# Standard library imports
from datetime import datetime
from typing import List, Dict, Optional
from pathlib import Path
from tqdm import tqdm
import os

# Additional imports
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    print("├─ ✅ Jupyter widgets")
except ImportError:
    print("├─ ⚠️ Jupyter widgets not available")

try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    print("├─ ✅ Plotly")
except ImportError:
    print("├─ ⚠️ Plotly not available")

# AI and ML imports with fallbacks
try:
    import insightface
    from insightface.app import FaceAnalysis
    print("├─ ✅ InsightFace")
    INSIGHTFACE_AVAILABLE = True
except ImportError:
    print("├─ ❌ InsightFace not available - using mock system")
    INSIGHTFACE_AVAILABLE = False

try:
    from sklearn.metrics.pairwise import cosine_similarity
    print("├─ ✅ Scikit-learn")
    SKLEARN_AVAILABLE = True
except ImportError:
    print("├─ ❌ Scikit-learn not available")
    SKLEARN_AVAILABLE = False

try:
    import psutil
    print("├─ ✅ PSUtil")
except ImportError:
    print("├─ ⚠️ PSUtil not available")

# Check critical dependencies
critical_modules = [cv2, np, sqlite3, time]
missing_critical = [i for i, module in enumerate(critical_modules) if module is None]

if missing_critical:
    print(f"\n❌ CRITICAL ERROR: Missing essential modules")
    print("Please restart kernel and run Cell 1 again, or install manually:")
    print("pip install opencv-python numpy")
    raise ImportError("Critical dependencies missing")

print("✅ Core modules loaded successfully")

# Directory setup
base_dir = Path.cwd()
employees_dir = base_dir / 'employees'
data_dir = base_dir / 'data'
snapshots_dir = base_dir / 'snapshots'
exports_dir = base_dir / 'exports'

# Initialize AI Models for Local Processing
class LocalAISystem:
    def __init__(self):
        print("🔄 Initializing Local AI System...")
        self.app = None
        self.performance_stats = {
            'total_inferences': 0,
            'avg_latency_ms': 0.0,
            'total_time': 0.0
        }
        self.mock_mode = not INSIGHTFACE_AVAILABLE
        
        if self.mock_mode:
            print("⚠️ Running in MOCK MODE (InsightFace not available)")
            self._init_mock_system()
        else:
            self._init_models()
    
    def _init_mock_system(self):
        """Initialize mock system for testing without InsightFace"""
        print("🔧 Mock AI system initialized")
        print("├─ Face detection: Simulated")
        print("├─ Face recognition: Random embeddings")
        print("└─ Performance: Fast but not functional")
    
    def _init_models(self):
        try:
            # Setup providers
            try:
                import torch
                if torch.cuda.is_available():
                    providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
                    print(f"🚀 GPU Mode: {torch.cuda.get_device_name(0)}")
                else:
                    providers = ['CPUExecutionProvider']
                    print("💻 CPU Mode: Slower but functional")
            except ImportError:
                providers = ['CPUExecutionProvider']
                print("💻 CPU Mode: PyTorch not available")
            
            # Initialize with lighter model for local demo
            model_pack = 'buffalo_s'  # Faster for local processing
            print(f"📦 Loading {model_pack} model pack...")
            
            self.app = FaceAnalysis(name=model_pack, providers=providers)
            ctx_id = 0 if 'CUDAExecutionProvider' in providers else -1
            self.app.prepare(ctx_id=ctx_id, det_size=(640, 640), det_thresh=0.5)
            
            print(f"✅ {model_pack} loaded successfully!")
            
        except Exception as e:
            print(f"❌ Model loading failed: {e}")
            print("🔄 Falling back to mock mode...")
            self.mock_mode = True
            self._init_mock_system()
    
    def detect_and_recognize(self, image):
        """Process image and return face data"""
        start_time = time.time()
        
        if self.mock_mode:
            # Mock implementation for testing
            if image is not None and image.size > 0:
                # Return mock face data
                height, width = image.shape[:2]
                mock_face = {
                    'bbox': [width*0.3, height*0.2, width*0.7, height*0.8],
                    'det_score': 0.95,
                    'landmarks': None,
                    'embedding': np.random.randn(512).astype(np.float32),
                    'age': np.random.randint(20, 60),
                    'gender': np.random.choice(['M', 'F'])
                }
                results = [mock_face]
            else:
                results = []
        else:
            try:
                faces = self.app.get(image)
                
                results = []
                for face in faces:
                    result = {
                        'bbox': face.bbox,
                        'det_score': face.det_score,
                        'landmarks': getattr(face, 'kps', None),
                        'embedding': face.embedding,
                        'age': getattr(face, 'age', None),
                        'gender': getattr(face, 'gender', None)
                    }
                    results.append(result)
            except Exception as e:
                print(f"AI processing error: {e}")
                results = []
        
        # Update performance stats
        processing_time = (time.time() - start_time) * 1000
        self.performance_stats['total_inferences'] += 1
        self.performance_stats['total_time'] += processing_time
        self.performance_stats['avg_latency_ms'] = (
            self.performance_stats['total_time'] / 
            self.performance_stats['total_inferences']
        )
        
        return results

# Initialize Local Database
class LocalDatabase:
    def __init__(self, db_path="local_attendance.db"):
        self.db_path = data_dir / db_path
        try:
            self.conn = sqlite3.connect(str(self.db_path), check_same_thread=False)
            self.conn.row_factory = sqlite3.Row
            self._create_tables()
            print(f"🗄️ Local database: {self.db_path}")
        except Exception as e:
            print(f"❌ Database initialization failed: {e}")
            raise
    
    def _create_tables(self):
        cursor = self.conn.cursor()
        
        # Employees table
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS employees (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            employee_code TEXT UNIQUE NOT NULL,
            name TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            department TEXT,
            position TEXT,
            face_embeddings TEXT,
            folder_path TEXT,
            face_count INTEGER DEFAULT 0,
            avg_quality REAL DEFAULT 0.0,
            is_active BOOLEAN DEFAULT 1,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        # Attendance logs
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS attendance_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            employee_id INTEGER,
            event_type TEXT NOT NULL,
            timestamp TIMESTAMP NOT NULL,
            confidence REAL NOT NULL,
            snapshot_path TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (employee_id) REFERENCES employees (id)
        )
        """)
        
        self.conn.commit()
    
    def register_employee(self, employee_data, face_embeddings, folder_path):
        """Register employee with face embeddings"""
        cursor = self.conn.cursor()
        
        try:
            # Calculate average embedding
            if face_embeddings:
                avg_embedding = np.mean(face_embeddings, axis=0)
                avg_quality = np.mean([self._calculate_quality(emb) for emb in face_embeddings])
            else:
                return None
            
            cursor.execute("""
            INSERT INTO employees 
            (employee_code, name, email, department, position, face_embeddings, 
             folder_path, face_count, avg_quality)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                employee_data['employee_code'],
                employee_data['name'],
                employee_data['email'],
                employee_data.get('department', ''),
                employee_data.get('position', ''),
                json.dumps(avg_embedding.tolist()),
                str(folder_path),
                len(face_embeddings),
                avg_quality
            ))
            
            employee_id = cursor.lastrowid
            self.conn.commit()
            
            return employee_id
            
        except Exception as e:
            self.conn.rollback()
            print(f"Registration error: {e}")
            return None
    
    def _calculate_quality(self, embedding):
        """Simple embedding quality metric"""
        return min(np.linalg.norm(embedding) / 1.0, 1.0)
    
    def find_employee_by_embedding(self, query_embedding, threshold=0.65):
        """Find employee by face embedding"""
        cursor = self.conn.cursor()
        
        cursor.execute("""
        SELECT id, employee_code, name, email, face_embeddings
        FROM employees WHERE is_active = 1 AND face_embeddings IS NOT NULL
        """)
        
        best_match = None
        best_similarity = 0.0
        
        for row in cursor.fetchall():
            try:
                stored_embedding = np.array(json.loads(row['face_embeddings']))
                
                if SKLEARN_AVAILABLE:
                    similarity = cosine_similarity(
                        query_embedding.reshape(1, -1),
                        stored_embedding.reshape(1, -1)
                    )[0][0]
                else:
                    # Fallback similarity calculation
                    similarity = np.dot(query_embedding, stored_embedding) / (
                        np.linalg.norm(query_embedding) * np.linalg.norm(stored_embedding)
                    )
                
                if similarity > best_similarity and similarity > threshold:
                    best_similarity = similarity
                    best_match = {
                        'id': row['id'],
                        'employee_code': row['employee_code'],
                        'name': row['name'],
                        'email': row['email'],
                        'similarity': similarity
                    }
            except:
                continue
        
        return best_match
    
    def record_attendance(self, employee_id, event_type, confidence, snapshot_path=None):
        """Record attendance event"""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        cursor = self.conn.cursor()
        cursor.execute("""
        INSERT INTO attendance_logs (employee_id, event_type, timestamp, confidence, snapshot_path)
        VALUES (?, ?, ?, ?, ?)
        """, (employee_id, event_type, timestamp, confidence, snapshot_path))
        
        self.conn.commit()
        return cursor.lastrowid
    
    def get_statistics(self):
        """Get database statistics"""
        cursor = self.conn.cursor()
        
        stats = {}
        
        cursor.execute("SELECT COUNT(*) FROM employees WHERE is_active = 1")
        stats['total_employees'] = cursor.fetchone()[0]
        
        cursor.execute("SELECT COUNT(*) FROM attendance_logs")
        stats['total_logs'] = cursor.fetchone()[0]
        
        cursor.execute("""
        SELECT COUNT(*) FROM attendance_logs 
        WHERE DATE(timestamp) = DATE('now')
        """)
        stats['today_logs'] = cursor.fetchone()[0]
        
        return stats

# Initialize systems with error handling
print("\n🔄 INITIALIZING SYSTEMS:")

try:
    ai_system = LocalAISystem()
    print("├─ ✅ AI System initialized")
except Exception as e:
    print(f"├─ ❌ AI System failed: {e}")
    ai_system = None

try:
    db = LocalDatabase()
    print("├─ ✅ Database initialized")
except Exception as e:
    print(f"├─ ❌ Database failed: {e}")
    db = None

if ai_system and db:
    print("└─ ✅ All systems ready!")
    
    print("\n📊 Current Database Statistics:")
    stats = db.get_statistics()
    print(f"├─ Employees: {stats['total_employees']}")
    print(f"├─ Total Logs: {stats['total_logs']}")
    print(f"└─ Today's Logs: {stats['today_logs']}")
    
    if ai_system.mock_mode:
        print("\n⚠️ IMPORTANT: Running in MOCK MODE")
        print("├─ Face detection/recognition simulated")
        print("├─ Install 'insightface' for real AI functionality")
        print("└─ Current setup good for testing workflow")
else:
    print("└─ ❌ System initialization incomplete")

print("\n💡 EMPLOYEE FOLDER SETUP:")
print(f"1. Create folders in: {employees_dir}")
print(f"2. Folder structure: employees/John_Doe/photo1.jpg, photo2.jpg, ...")
print(f"3. Run the next cell to auto-scan and register employees")
print("=" * 35)

🤖 AI SYSTEM & DATABASE SETUP
📦 LOADING REQUIRED MODULES:
├─ ✅ Jupyter widgets
├─ ✅ Plotly
├─ ✅ InsightFace
├─ ✅ Scikit-learn
├─ ✅ PSUtil
✅ Core modules loaded successfully

🔄 INITIALIZING SYSTEMS:
🔄 Initializing Local AI System...
💻 CPU Mode: Slower but functional
📦 Loading buffalo_s model pack...
download_path: C:\Users\hoang/.insightface\models\buffalo_s


100%|██████████| 124617/124617 [00:50<00:00, 2477.49KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hoang/.insightface\models\buffalo_s\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hoang/.insightface\models\buffalo_s\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hoang/.insightface\models\buffalo_s\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hoang/.insightface\models\buffalo_s\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\hoang/.insightface\models\buffalo_s\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127

## Cell 3: 📁 Employee Registration & Camera Setup

**Smart employee folder scanning and camera initialization**


In [7]:
# Cell 3: 📁 Employee Registration & Camera Setup - IMPROVED VERSION
# Employee Registration & Camera Setup with Enhanced Error Handling

import os
import glob
from pathlib import Path
import threading
import queue
import time
from collections import defaultdict
import platform

print("📁 EMPLOYEE REGISTRATION & CAMERA SETUP")
print("=" * 45)

class LocalEmployeeManager:
    """Smart employee folder scanning and registration with error handling"""
    
    def __init__(self, ai_system, database):
        self.ai_system = ai_system
        self.db = database
        self.registered_employees = {}
        self.scan_results = []
        
        # Check system availability
        if ai_system is None or database is None:
            print("❌ AI System or Database not available")
            print("Please run Cell 2 successfully first")
            self.available = False
        else:
            self.available = True
    
    def scan_employee_folders(self, auto_register=True):
        """Scan and optionally auto-register employees from folders"""
        if not self.available:
            print("❌ System not available - check Cell 2")
            return
            
        print(f"🔍 Scanning employee folders in: {employees_dir}")
        
        if not employees_dir.exists():
            employees_dir.mkdir(parents=True, exist_ok=True)
            print(f"✅ Created employees directory: {employees_dir}")
            self._create_sample_instructions()
            return
        
        # Find employee folders (ignore hidden and sample folders)
        employee_folders = [f for f in employees_dir.iterdir() 
                           if f.is_dir() 
                           and not f.name.startswith('.') 
                           and not f.name.startswith('_')]
        
        if not employee_folders:
            print("⚠️ No employee folders found")
            self._create_sample_instructions()
            return
        
        print(f"📁 Found {len(employee_folders)} employee folders:")
        for folder in employee_folders:
            print(f"  ├─ {folder.name}")
        
        # Process each folder with progress tracking
        total_registered = 0
        self.scan_results.clear()
        
        print(f"\n🔄 Processing employee folders...")
        
        for folder in tqdm(employee_folders, desc="Processing employees"):
            try:
                result = self._process_employee_folder(folder, auto_register)
                if result:
                    self.scan_results.append(result)
                    if result['registered']:
                        total_registered += 1
            except Exception as e:
                print(f"❌ Error processing {folder.name}: {e}")
        
        print(f"\n✅ Scan completed:")
        print(f"├─ Folders processed: {len(employee_folders)}")
        print(f"├─ Successfully registered: {total_registered}")
        print(f"└─ Database employees: {self.db.get_statistics()['total_employees']}")
        
        # Show summary table
        if self.scan_results:
            self._show_scan_summary()
    
    def _create_sample_instructions(self):
        """Create sample folder structure and instructions"""
        print("💡 Creating sample employee folder structure...")
        
        sample_folders = ['John_Doe', 'Jane_Smith', 'Michael_Johnson']
        
        for folder_name in sample_folders:
            sample_folder = employees_dir / folder_name
            sample_folder.mkdir(exist_ok=True)
            
            # Create README in each folder
            readme_content = f"""# Employee: {folder_name.replace('_', ' ')}

## Instructions:
1. Add 1-5 clear photos of this employee
2. Use formats: .jpg, .jpeg, .png
3. Ensure good lighting and clear face view
4. Avoid group photos - one person per photo

## Example files:
- photo1.jpg
- photo2.jpg
- headshot.png
"""
            with open(sample_folder / "README.md", "w") as f:
                f.write(readme_content)
        
        print(f"├─ Sample folders created: {', '.join(sample_folders)}")
        print(f"└─ Add photos to these folders, then run this cell again")
    
    def _process_employee_folder(self, folder_path, auto_register=True):
        """Process single employee folder with comprehensive error handling"""
        employee_name = folder_path.name.replace('_', ' ').title()
        
        # Find image files
        image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']
        image_files = []
        
        for ext in image_extensions:
            image_files.extend(folder_path.glob(ext))
        
        if not image_files:
            print(f"  ⚠️ {employee_name}: No images found")
            return {
                'name': employee_name,
                'folder_path': str(folder_path),
                'total_images': 0,
                'valid_images': 0,
                'avg_quality': 0,
                'registered': False,
                'error': 'No images found'
            }
        
        # Process images with error handling
        face_embeddings = []
        quality_scores = []
        valid_images = 0
        processing_errors = []
        
        for img_file in image_files:
            try:
                # Load image with error handling
                image = cv2.imread(str(img_file))
                if image is None:
                    processing_errors.append(f"Cannot read {img_file.name}")
                    continue
                
                # Check image size
                if image.shape[0] < 100 or image.shape[1] < 100:
                    processing_errors.append(f"Image too small: {img_file.name}")
                    continue
                
                # Detect faces
                faces = self.ai_system.detect_and_recognize(image)
                
                if len(faces) == 1:  # Exactly one face
                    face_data = faces[0]
                    
                    # Quality threshold check
                    det_score = face_data['det_score']
                    if det_score > 0.6:  # Good quality threshold
                        face_embeddings.append(face_data['embedding'])
                        quality_scores.append(det_score)
                        valid_images += 1
                    else:
                        processing_errors.append(f"Low quality face in {img_file.name}")
                elif len(faces) == 0:
                    processing_errors.append(f"No face detected in {img_file.name}")
                else:
                    processing_errors.append(f"Multiple faces in {img_file.name}")
                
            except Exception as e:
                processing_errors.append(f"Error processing {img_file.name}: {str(e)}")
                continue
        
        # Create result summary
        result = {
            'name': employee_name,
            'folder_path': str(folder_path),
            'total_images': len(image_files),
            'valid_images': valid_images,
            'avg_quality': np.mean(quality_scores) if quality_scores else 0,
            'registered': False,
            'employee_id': None,
            'processing_errors': processing_errors
        }
        
        # Auto-register if sufficient quality faces
        if auto_register and len(face_embeddings) >= 1:
            try:
                employee_data = {
                    'employee_code': employee_name.upper().replace(' ', '_'),
                    'name': employee_name,
                    'email': f"{employee_name.lower().replace(' ', '.')}@company.com",
                    'department': 'Auto-Registered',
                    'position': 'Employee'
                }
                
                employee_id = self.db.register_employee(
                    employee_data, face_embeddings, folder_path
                )
                
                if employee_id:
                    result['registered'] = True
                    result['employee_id'] = employee_id
                    self.registered_employees[employee_id] = result
                    print(f"  ✅ {employee_name}: Registered ({valid_images} faces, avg quality: {result['avg_quality']:.3f})")
                else:
                    result['error'] = 'Database registration failed'
                    print(f"  ❌ {employee_name}: Registration failed")
            except Exception as e:
                result['error'] = f'Registration error: {str(e)}'
                print(f"  ❌ {employee_name}: Registration error - {str(e)}")
        else:
            reason = f"Insufficient quality faces ({valid_images})" if len(face_embeddings) < 1 else "Auto-register disabled"
            result['error'] = reason
            print(f"  ⚠️ {employee_name}: {reason}")
        
        return result
    
    def _show_scan_summary(self):
        """Show employee scan summary table"""
        print(f"\n📋 EMPLOYEE SCAN SUMMARY")
        print("=" * 80)
        
        # Create summary data
        summary_data = []
        for result in self.scan_results:
            error_msg = result.get('error', '')
            if len(error_msg) > 30:
                error_msg = error_msg[:27] + "..."
            
            summary_data.append({
                'Employee': result['name'],
                'Total Images': result['total_images'],
                'Valid Faces': result['valid_images'],
                'Avg Quality': f"{result['avg_quality']:.3f}",
                'Registered': '✅' if result['registered'] else '❌',
                'Employee ID': result['employee_id'] or 'N/A',
                'Status': 'Success' if result['registered'] else error_msg
            })
        
        # Display as formatted table
        if summary_data:
            df = pd.DataFrame(summary_data)
            try:
                display(HTML(df.to_html(index=False, escape=False)))
            except:
                # Fallback to simple print if HTML display fails
                print(df.to_string(index=False))
        
        # Show detailed errors if any
        error_details = []
        for result in self.scan_results:
            if result.get('processing_errors'):
                error_details.append({
                    'employee': result['name'],
                    'errors': result['processing_errors']
                })
        
        if error_details:
            print(f"\n⚠️ DETAILED ERROR REPORT:")
            for detail in error_details:
                print(f"\n{detail['employee']}:")
                for error in detail['errors']:
                    print(f"  ├─ {error}")
    
    def manual_add_employee(self, name, folder_path):
        """Manually add employee from specific folder"""
        if not self.available:
            print("❌ System not available")
            return None
            
        folder = Path(folder_path)
        if not folder.exists():
            print(f"❌ Folder not found: {folder_path}")
            return None
        
        print(f"👤 Processing: {name}")
        result = self._process_employee_folder(folder, auto_register=True)
        
        if result and result['registered']:
            print(f"✅ Successfully registered: {name}")
            return result['employee_id']
        else:
            print(f"❌ Failed to register: {name}")
            if result and result.get('error'):
                print(f"   Reason: {result['error']}")
            return None

class LocalCameraManager:
    """Enhanced camera management for local demo with better error handling"""
    
    def __init__(self):
        self.camera = None
        self.camera_active = False
        self.camera_thread = None
        self.frame_queue = queue.Queue(maxsize=10)
        self.latest_frame = None
        self.camera_stats = {
            'frames_captured': 0,
            'frames_processed': 0,
            'fps': 0,
            'last_fps_time': time.time()
        }
        self.available_cameras = []
    
    def detect_cameras(self):
        """Detect available cameras with comprehensive testing"""
        print("🔍 Detecting available cameras...")
        
        self.available_cameras = []
        
        # Test different camera indices based on platform
        max_cameras = 10 if platform.system() == "Windows" else 4
        
        for i in range(max_cameras):
            try:
                print(f"  Testing camera {i}...", end="")
                cap = cv2.VideoCapture(i)
                
                if cap.isOpened():
                    # Try to read a frame
                    ret, frame = cap.read()
                    if ret and frame is not None and frame.size > 0:
                        # Get camera properties
                        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        fps = cap.get(cv2.CAP_PROP_FPS)
                        
                        camera_info = {
                            'index': i,
                            'name': f'Camera {i}',
                            'resolution': f'{width}x{height}',
                            'fps': fps,
                            'working': True,
                            'test_frame_shape': frame.shape
                        }
                        
                        self.available_cameras.append(camera_info)
                        print(f" ✅ {width}x{height} @ {fps:.1f} FPS")
                    else:
                        print(f" ❌ Cannot capture frame")
                else:
                    print(f" ❌ Cannot open")
                
                cap.release()
                
            except Exception as e:
                print(f" ❌ Error: {str(e)[:30]}")
                continue
        
        print(f"\n📊 Camera Detection Results:")
        if self.available_cameras:
            print(f"├─ Found {len(self.available_cameras)} working cameras")
            for cam in self.available_cameras:
                print(f"├─ Camera {cam['index']}: {cam['resolution']} @ {cam['fps']:.1f} FPS")
            print(f"└─ Ready for camera initialization")
        else:
            print(f"├─ No working cameras found")
            print(f"├─ Possible solutions:")
            print(f"│  ├─ Connect a USB webcam")
            print(f"│  ├─ Check camera permissions")
            print(f"│  ├─ Close other applications using camera")
            print(f"│  └─ Restart computer if camera was recently connected")
            print(f"└─ You can still test with image files")
        
        return self.available_cameras
    
    def initialize_camera(self, camera_index=0):
        """Initialize specific camera with enhanced error handling"""
        print(f"📷 Initializing camera {camera_index}...")
        
        # Check if camera was detected
        if not any(cam['index'] == camera_index for cam in self.available_cameras):
            print(f"❌ Camera {camera_index} not in detected cameras list")
            print(f"Available cameras: {[cam['index'] for cam in self.available_cameras]}")
            return False
        
        try:
            # Clean up existing camera
            if self.camera:
                self.camera.release()
            
            # Initialize new camera
            self.camera = cv2.VideoCapture(camera_index)
            
            if not self.camera.isOpened():
                print(f"❌ Cannot open camera {camera_index}")
                return False
            
            # Set optimal settings with error handling
            settings = [
                (cv2.CAP_PROP_FRAME_WIDTH, 640),
                (cv2.CAP_PROP_FRAME_HEIGHT, 480),
                (cv2.CAP_PROP_FPS, 30),
                (cv2.CAP_PROP_BUFFERSIZE, 1)
            ]
            
            for prop, value in settings:
                try:
                    self.camera.set(prop, value)
                except:
                    pass  # Some cameras don't support all properties
            
            # Test capture with multiple attempts
            test_successful = False
            for attempt in range(3):
                ret, test_frame = self.camera.read()
                if ret and test_frame is not None and test_frame.size > 0:
                    test_successful = True
                    break
                time.sleep(0.1)
            
            if not test_successful:
                print(f"❌ Cannot capture test frame after 3 attempts")
                self.camera.release()
                self.camera = None
                return False
            
            # Get actual settings
            actual_width = int(self.camera.get(cv2.CAP_PROP_FRAME_WIDTH))
            actual_height = int(self.camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
            actual_fps = self.camera.get(cv2.CAP_PROP_FPS)
            
            print(f"✅ Camera initialized successfully:")
            print(f"├─ Resolution: {actual_width}x{actual_height}")
            print(f"├─ Target FPS: {actual_fps}")
            print(f"├─ Test frame shape: {test_frame.shape}")
            print(f"└─ Status: Ready for capture")
            
            return True
            
        except Exception as e:
            print(f"❌ Camera initialization error: {e}")
            if self.camera:
                self.camera.release()
                self.camera = None
            return False
    
    def start_camera_thread(self):
        """Start threaded camera capture with error handling"""
        if not self.camera or not self.camera.isOpened():
            print("❌ Camera not initialized")
            return False
        
        if self.camera_active:
            print("⚠️ Camera thread already running")
            return True
        
        try:
            self.camera_active = True
            self.camera_thread = threading.Thread(target=self._camera_capture_loop, daemon=True)
            self.camera_thread.start()
            
            # Wait a moment and check if thread is working
            time.sleep(0.5)
            if self.camera_thread.is_alive():
                print("🎥 Camera thread started successfully")
                return True
            else:
                print("❌ Camera thread failed to start")
                self.camera_active = False
                return False
                
        except Exception as e:
            print(f"❌ Camera thread start error: {e}")
            self.camera_active = False
            return False
    
    def _camera_capture_loop(self):
        """Camera capture loop with robust error handling"""
        fps_counter = 0
        fps_start_time = time.time()
        consecutive_failures = 0
        max_failures = 10
        
        print("🔄 Camera capture loop started")
        
        while self.camera_active and self.camera and self.camera.isOpened():
            try:
                ret, frame = self.camera.read()
                
                if ret and frame is not None and frame.size > 0:
                    self.latest_frame = frame.copy()
                    self.camera_stats['frames_captured'] += 1
                    consecutive_failures = 0
                    
                    # Update frame queue (non-blocking)
                    try:
                        self.frame_queue.put_nowait(frame)
                    except queue.Full:
                        # Remove oldest frame if queue is full
                        try:
                            self.frame_queue.get_nowait()
                            self.frame_queue.put_nowait(frame)
                        except queue.Empty:
                            pass
                    
                    # Calculate FPS
                    fps_counter += 1
                    if fps_counter >= 30:  # Update every 30 frames
                        current_time = time.time()
                        elapsed = current_time - fps_start_time
                        if elapsed > 0:
                            self.camera_stats['fps'] = fps_counter / elapsed
                        fps_counter = 0
                        fps_start_time = current_time
                
                else:
                    consecutive_failures += 1
                    if consecutive_failures >= max_failures:
                        print(f"❌ Too many consecutive capture failures ({max_failures})")
                        break
                
                time.sleep(0.001)  # Small delay to prevent busy waiting
                
            except Exception as e:
                consecutive_failures += 1
                print(f"❌ Camera capture error: {e}")
                if consecutive_failures >= max_failures:
                    print(f"❌ Too many capture errors, stopping camera thread")
                    break
                time.sleep(0.1)  # Longer delay after error
        
        print("🛑 Camera capture loop ended")
        self.camera_active = False
    
    def get_latest_frame(self):
        """Get the most recent frame"""
        return self.latest_frame
    
    def stop_camera(self):
        """Stop camera capture with cleanup"""
        print("🛑 Stopping camera...")
        self.camera_active = False
        
        if self.camera_thread and self.camera_thread.is_alive():
            self.camera_thread.join(timeout=3.0)
            if self.camera_thread.is_alive():
                print("⚠️ Camera thread did not stop gracefully")
        
        if self.camera:
            self.camera.release()
            self.camera = None
        
        # Clear frame queue
        while not self.frame_queue.empty():
            try:
                self.frame_queue.get_nowait()
            except queue.Empty:
                break
        
        print("✅ Camera stopped and cleaned up")
    
    def get_camera_stats(self):
        """Get camera performance statistics"""
        return self.camera_stats.copy()
    
    def test_camera_functionality(self, camera_index=0, duration_seconds=5):
        """Test camera functionality comprehensively"""
        print(f"🧪 Testing camera {camera_index} for {duration_seconds} seconds...")
        
        if not self.initialize_camera(camera_index):
            return False
        
        if not self.start_camera_thread():
            return False
        
        start_time = time.time()
        last_frame_count = 0
        
        try:
            while time.time() - start_time < duration_seconds:
                current_frame_count = self.camera_stats['frames_captured']
                current_fps = self.camera_stats['fps']
                
                print(f"\r  Frames: {current_frame_count}, FPS: {current_fps:.1f}", end="")
                time.sleep(0.5)
            
            final_frames = self.camera_stats['frames_captured']
            final_fps = self.camera_stats['fps']
            
            print(f"\n✅ Camera test completed:")
            print(f"├─ Total frames captured: {final_frames}")
            print(f"├─ Average FPS: {final_fps:.1f}")
            print(f"├─ Frame rate: {'Good' if final_fps > 15 else 'Low'}")
            print(f"└─ Status: {'✅ Working' if final_frames > 10 else '❌ Issues detected'}")
            
            return final_frames > 10
            
        except KeyboardInterrupt:
            print(f"\n⏹️ Camera test interrupted")
            return False
        finally:
            self.stop_camera()

# Initialize managers with availability checks
print("\n🔄 INITIALIZING MANAGERS:")

try:
    employee_manager = LocalEmployeeManager(ai_system, db)
    if employee_manager.available:
        print("├─ ✅ Employee Manager ready")
    else:
        print("├─ ❌ Employee Manager unavailable (check AI system and database)")
except Exception as e:
    print(f"├─ ❌ Employee Manager error: {e}")
    employee_manager = None

try:
    camera_manager = LocalCameraManager()
    print("├─ ✅ Camera Manager ready")
except Exception as e:
    print(f"├─ ❌ Camera Manager error: {e}")
    camera_manager = None

print("└─ Manager initialization completed")

print("\n🎯 AVAILABLE ACTIONS:")
if employee_manager and employee_manager.available:
    print("📁 Employee Management:")
    print("├─ employee_manager.scan_employee_folders()")
    print("├─ employee_manager.manual_add_employee('Name', 'path/to/folder')")
    print("└─ Check employees/ folder structure")
else:
    print("❌ Employee management not available")

if camera_manager:
    print("\n📷 Camera Management:")
    print("├─ camera_manager.detect_cameras()")
    print("├─ camera_manager.initialize_camera(0)")
    print("├─ camera_manager.test_camera_functionality(0)")
    print("└─ camera_manager.start_camera_thread()")
else:
    print("❌ Camera management not available")

print("\n💡 RECOMMENDED WORKFLOW:")
print("1. First: employee_manager.scan_employee_folders()")
print("2. Then: camera_manager.detect_cameras()")
print("3. Finally: Run Cell 4 for real-time processing")
print("=" * 45)

📁 EMPLOYEE REGISTRATION & CAMERA SETUP

🔄 INITIALIZING MANAGERS:
├─ ✅ Employee Manager ready
├─ ✅ Camera Manager ready
└─ Manager initialization completed

🎯 AVAILABLE ACTIONS:
📁 Employee Management:
├─ employee_manager.scan_employee_folders()
├─ employee_manager.manual_add_employee('Name', 'path/to/folder')
└─ Check employees/ folder structure

📷 Camera Management:
├─ camera_manager.detect_cameras()
├─ camera_manager.initialize_camera(0)
├─ camera_manager.test_camera_functionality(0)
└─ camera_manager.start_camera_thread()

💡 RECOMMENDED WORKFLOW:
1. First: employee_manager.scan_employee_folders()
2. Then: camera_manager.detect_cameras()
3. Finally: Run Cell 4 for real-time processing


## Cell 4: 🎥 Real-time Camera Processing

**Live camera feed with attendance tracking and performance monitoring**


In [8]:
# Cell 4: 🎥 Real-time Camera Processing - IMPROVED VERSION
# Real-time Camera Processing with Enhanced Error Handling and Monitoring

import cv2
import numpy as np
import time
from collections import deque, defaultdict
from datetime import datetime
import threading
import queue
import json

print("🎥 REAL-TIME CAMERA PROCESSING")
print("=" * 35)

class AttendanceProcessor:
    """Real-time attendance processing with comprehensive error handling"""
    
    def __init__(self, ai_system, database, camera_manager):
        self.ai_system = ai_system
        self.db = database
        self.camera_manager = camera_manager
        
        # Check system availability
        if not all([ai_system, database, camera_manager]):
            print("❌ Required systems not available")
            self.available = False
            return
        
        if hasattr(ai_system, 'mock_mode') and ai_system.mock_mode:
            print("⚠️ Running with MOCK AI system - for testing only")
        
        self.available = True
        
        # Processing settings
        self.processing_active = False
        self.process_every_n_frames = 3  # Process every 3rd frame for performance
        self.frame_counter = 0
        
        # Recognition history and cooldown
        self.recognition_history = deque(maxlen=100)
        self.employee_cooldowns = {}  # employee_id -> last_recognition_time
        self.cooldown_period = 30  # seconds
        
        # Statistics with error tracking
        self.stats = {
            'total_frames': 0,
            'processed_frames': 0,
            'faces_detected': 0,
            'employees_recognized': 0,
            'processing_times': deque(maxlen=50),
            'session_start': time.time(),
            'errors': {
                'camera_errors': 0,
                'ai_errors': 0,
                'db_errors': 0,
                'general_errors': 0
            }
        }
        
        # Attendance events
        self.attendance_events = []
        
        # Display and monitoring
        self.last_display_update = 0
        self.display_update_interval = 2.0  # seconds
        
        print("✅ Attendance Processor initialized")
    
    def start_real_time_processing(self, duration_minutes=5, display_video=False, 
                                  show_annotated_frames=False):
        """Start real-time attendance processing with enhanced monitoring"""
        
        if not self.available:
            print("❌ Processor not available - check previous cells")
            return
        
        print(f"🚀 Starting real-time processing for {duration_minutes} minutes...")
        
        # Initialize camera if needed
        if not self._ensure_camera_ready():
            return
        
        # Reset statistics
        self._reset_session_stats()
        
        print("📊 Processing started - Press Ctrl+C in notebook to stop early")
        print("=" * 60)
        
        try:
            end_time = time.time() + (duration_minutes * 60)
            
            while self.processing_active and time.time() < end_time:
                try:
                    # Get latest frame
                    frame = self.camera_manager.get_latest_frame()
                    
                    if frame is not None:
                        self.stats['total_frames'] += 1
                        self.frame_counter += 1
                        
                        # Process frame (skip frames for performance)
                        if self.frame_counter % self.process_every_n_frames == 0:
                            self._process_frame_safe(frame)
                        
                        # Show annotated frame if requested
                        if show_annotated_frames and hasattr(self, 'last_processed_frame'):
                            self._display_annotated_frame()
                        
                        # Update display periodically
                        current_time = time.time()
                        if current_time - self.last_display_update >= self.display_update_interval:
                            self._update_real_time_display()
                            self.last_display_update = current_time
                    else:
                        # Handle camera disconnection
                        self.stats['errors']['camera_errors'] += 1
                        if self.stats['errors']['camera_errors'] > 10:
                            print("❌ Too many camera errors - stopping processing")
                            break
                    
                    time.sleep(0.01)  # Small delay to prevent excessive CPU usage
                    
                except KeyboardInterrupt:
                    print("\n⏹️ Processing stopped by user (Ctrl+C)")
                    break
                except Exception as e:
                    self.stats['errors']['general_errors'] += 1
                    print(f"\n❌ Processing error: {e}")
                    if self.stats['errors']['general_errors'] > 5:
                        print("❌ Too many general errors - stopping processing")
                        break
                    time.sleep(1)  # Brief pause after error
            
        except Exception as e:
            print(f"\n❌ Fatal processing error: {e}")
        finally:
            self.processing_active = False
            self._cleanup_processing()
            print("\n📊 PROCESSING SUMMARY")
            self._show_session_summary()
    
    def _ensure_camera_ready(self):
        """Ensure camera is ready for processing"""
        try:
            # Check if camera is already running
            if self.camera_manager.camera_active:
                print("✅ Camera already active")
                return True
            
            # Detect cameras if not done
            if not self.camera_manager.available_cameras:
                print("🔍 Detecting cameras...")
                cameras = self.camera_manager.detect_cameras()
                if not cameras:
                    print("❌ No cameras detected")
                    print("💡 Connect a camera and run camera_manager.detect_cameras()")
                    return False
            
            # Initialize camera
            if not self.camera_manager.camera:
                print("📷 Initializing camera...")
                if not self.camera_manager.initialize_camera(0):
                    print("❌ Camera initialization failed")
                    return False
            
            # Start camera thread
            if not self.camera_manager.camera_active:
                print("🎥 Starting camera thread...")
                if not self.camera_manager.start_camera_thread():
                    print("❌ Camera thread start failed")
                    return False
            
            # Wait for first frame
            print("⏳ Waiting for camera warm-up...")
            for i in range(10):  # Wait up to 5 seconds
                if self.camera_manager.get_latest_frame() is not None:
                    print("✅ Camera ready!")
                    return True
                time.sleep(0.5)
            
            print("❌ Camera warm-up timeout")
            return False
            
        except Exception as e:
            print(f"❌ Camera setup error: {e}")
            return False
    
    def _reset_session_stats(self):
        """Reset statistics for new session"""
        self.processing_active = True
        self.stats = {
            'total_frames': 0,
            'processed_frames': 0,
            'faces_detected': 0,
            'employees_recognized': 0,
            'processing_times': deque(maxlen=50),
            'session_start': time.time(),
            'errors': {
                'camera_errors': 0,
                'ai_errors': 0,
                'db_errors': 0,
                'general_errors': 0
            }
        }
        self.attendance_events.clear()
        self.recognition_history.clear()
        self.employee_cooldowns.clear()
    
    def _process_frame_safe(self, frame):
        """Process frame with comprehensive error handling"""
        start_time = time.time()
        
        try:
            # AI processing with error handling
            try:
                faces = self.ai_system.detect_and_recognize(frame)
                self.stats['processed_frames'] += 1
            except Exception as e:
                self.stats['errors']['ai_errors'] += 1
                print(f"AI processing error: {e}")
                return
            
            processing_time = (time.time() - start_time) * 1000
            self.stats['processing_times'].append(processing_time)
            
            if faces:
                self.stats['faces_detected'] += 1
                
                # Store frame for annotation display
                self.last_processed_frame = {
                    'frame': frame.copy(),
                    'faces': faces,
                    'timestamp': time.time()
                }
                
                for face in faces:
                    self._process_detected_face_safe(face, frame)
        
        except Exception as e:
            self.stats['errors']['general_errors'] += 1
            print(f"Frame processing error: {e}")
    
    def _process_detected_face_safe(self, face_data, frame):
        """Process detected face with database error handling"""
        try:
            # Try to recognize employee
            employee = self.db.find_employee_by_embedding(
                face_data['embedding'], threshold=0.65
            )
            
            current_time = time.time()
            
            if employee:
                employee_id = employee['id']
                
                # Check cooldown period
                last_recognition = self.employee_cooldowns.get(employee_id, 0)
                if current_time - last_recognition >= self.cooldown_period:
                    
                    try:
                        # Record attendance
                        attendance_id = self.db.record_attendance(
                            employee_id, 'camera_checkin', employee['similarity']
                        )
                        
                        # Update cooldown
                        self.employee_cooldowns[employee_id] = current_time
                        
                        # Create attendance event
                        event = {
                            'attendance_id': attendance_id,
                            'employee_id': employee_id,
                            'employee_name': employee['name'],
                            'timestamp': current_time,
                            'confidence': employee['similarity'],
                            'bbox': face_data['bbox'],
                            'det_score': face_data['det_score']
                        }
                        
                        self.attendance_events.append(event)
                        self.stats['employees_recognized'] += 1
                        
                        # Add to recognition history
                        self.recognition_history.append({
                            'timestamp': current_time,
                            'employee_name': employee['name'],
                            'confidence': employee['similarity']
                        })
                        
                        # Show immediate notification
                        self._show_recognition_notification(event)
                        
                    except Exception as e:
                        self.stats['errors']['db_errors'] += 1
                        print(f"Database error: {e}")
        
        except Exception as e:
            self.stats['errors']['db_errors'] += 1
            print(f"Employee recognition error: {e}")
    
    def _show_recognition_notification(self, event):
        """Show immediate recognition notification"""
        timestamp_str = datetime.fromtimestamp(event['timestamp']).strftime('%H:%M:%S')
        print(f"\n🎉 {timestamp_str} - {event['employee_name']} recognized!")
        print(f"   Confidence: {event['confidence']:.3f} | ID: {event['attendance_id']}")
    
    def _update_real_time_display(self):
        """Update real-time display with current statistics"""
        # Calculate current stats
        session_duration = time.time() - self.stats['session_start']
        avg_processing_time = np.mean(self.stats['processing_times']) if self.stats['processing_times'] else 0
        processing_fps = 1000 / avg_processing_time if avg_processing_time > 0 else 0
        camera_fps = self.camera_manager.get_camera_stats()['fps']
        
        # Recent recognitions (last 60 seconds)
        recent_recognitions = [r for r in self.recognition_history if time.time() - r['timestamp'] < 60]
        
        # Clear output and display current status
        clear_output(wait=True)
        print(f"🎥 REAL-TIME ATTENDANCE PROCESSING")
        print(f"⏱️ Session Duration: {session_duration/60:.1f} minutes")
        print("=" * 50)
        
        print(f"📊 Performance Metrics:")
        print(f"├─ Camera FPS: {camera_fps:.1f}")
        print(f"├─ Processing FPS: {processing_fps:.1f}")
        print(f"├─ Avg Processing Time: {avg_processing_time:.1f}ms")
        print(f"└─ Frame Skip Ratio: 1:{self.process_every_n_frames}")
        
        print(f"\n🔍 Detection Statistics:")
        print(f"├─ Total Frames: {self.stats['total_frames']:,}")
        print(f"├─ Processed Frames: {self.stats['processed_frames']:,}")
        print(f"├─ Faces Detected: {self.stats['faces_detected']:,}")
        print(f"└─ Employees Recognized: {self.stats['employees_recognized']}")
        
        # Error summary
        total_errors = sum(self.stats['errors'].values())
        if total_errors > 0:
            print(f"\n⚠️ Error Summary:")
            for error_type, count in self.stats['errors'].items():
                if count > 0:
                    print(f"├─ {error_type.replace('_', ' ').title()}: {count}")
        
        if recent_recognitions:
            print(f"\n👥 Recent Recognitions (last 60s):")
            for i, rec in enumerate(recent_recognitions[-5:], 1):  # Show last 5
                timestamp_str = datetime.fromtimestamp(rec['timestamp']).strftime('%H:%M:%S')
                print(f"├─ {i}. {timestamp_str} - {rec['employee_name']} ({rec['confidence']:.3f})")
        
        active_cooldowns = len([c for c in self.employee_cooldowns.values() if time.time() - c < self.cooldown_period])
        print(f"\n💡 Active Cooldowns: {active_cooldowns}")
        print("=" * 50)
    
    def _display_annotated_frame(self):
        """Display annotated frame with face detection results"""
        try:
            if hasattr(self, 'last_processed_frame'):
                frame_data = self.last_processed_frame
                frame = frame_data['frame'].copy()
                faces = frame_data['faces']
                
                # Draw annotations
                for face in faces:
                    bbox = face['bbox']
                    x1, y1, x2, y2 = [int(x) for x in bbox[:4]]
                    
                    # Draw bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    
                    # Draw confidence score
                    conf_text = f"Det: {face['det_score']:.2f}"
                    cv2.putText(frame, conf_text, (x1, y1-10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                
                # Display frame (in Jupyter, you might want to save and show as image)
                # For now, we'll skip actual display to avoid issues
                pass
        except Exception as e:
            print(f"Display error: {e}")
    
    def _cleanup_processing(self):
        """Cleanup after processing session"""
        try:
            # Note: We don't stop camera here as it might be used again
            print("🧹 Processing cleanup completed")
        except Exception as e:
            print(f"Cleanup error: {e}")
    
    def _show_session_summary(self):
        """Show comprehensive session summary with error analysis"""
        session_duration = time.time() - self.stats['session_start']
        
        print("=" * 60)
        print(f"📈 SESSION SUMMARY")
        print(f"⏱️ Duration: {session_duration/60:.1f} minutes ({session_duration:.0f} seconds)")
        print("-" * 40)
        
        print(f"🎥 Video Processing:")
        print(f"├─ Total Frames: {self.stats['total_frames']:,}")
        print(f"├─ Processed Frames: {self.stats['processed_frames']:,}")
        processing_rate = self.stats['processed_frames']/session_duration if session_duration > 0 else 0
        print(f"├─ Processing Rate: {processing_rate:.1f} FPS")
        skip_ratio = self.stats['total_frames']/max(1, self.stats['processed_frames'])
        print(f"└─ Skip Ratio: {skip_ratio:.1f}x")
        
        print(f"\n🔍 Detection Results:")
        print(f"├─ Faces Detected: {self.stats['faces_detected']:,}")
        print(f"├─ Employees Recognized: {self.stats['employees_recognized']}")
        unique_employees = len(set(e['employee_id'] for e in self.attendance_events))
        print(f"├─ Unique Employees: {unique_employees}")
        recognition_rate = self.stats['employees_recognized']/max(1, self.stats['faces_detected'])*100
        print(f"└─ Recognition Rate: {recognition_rate:.1f}%")
        
        # Performance analysis
        if self.stats['processing_times']:
            avg_time = np.mean(self.stats['processing_times'])
            min_time = np.min(self.stats['processing_times'])
            max_time = np.max(self.stats['processing_times'])
            
            print(f"\n⚡ Performance Analysis:")
            print(f"├─ Avg Processing Time: {avg_time:.1f}ms")
            print(f"├─ Min Processing Time: {min_time:.1f}ms")
            print(f"├─ Max Processing Time: {max_time:.1f}ms")
            print(f"└─ Theoretical Max FPS: {1000/avg_time:.1f}")
        
        # Error analysis
        total_errors = sum(self.stats['errors'].values())
        if total_errors > 0:
            print(f"\n❌ Error Analysis:")
            print(f"├─ Total Errors: {total_errors}")
            for error_type, count in self.stats['errors'].items():
                if count > 0:
                    percentage = (count / max(1, self.stats['processed_frames'])) * 100
                    print(f"├─ {error_type.replace('_', ' ').title()}: {count} ({percentage:.1f}%)")
            
            # Error rate assessment
            error_rate = total_errors / max(1, self.stats['processed_frames']) * 100
            if error_rate < 1:
                print(f"└─ Error Rate: {error_rate:.2f}% (✅ Excellent)")
            elif error_rate < 5:
                print(f"└─ Error Rate: {error_rate:.1f}% (👍 Good)")
            elif error_rate < 10:
                print(f"└─ Error Rate: {error_rate:.1f}% (⚠️ Needs attention)")
            else:
                print(f"└─ Error Rate: {error_rate:.1f}% (❌ Poor - investigate)")
        
        # Attendance events summary
        if self.attendance_events:
            print(f"\n👥 Attendance Events:")
            employee_counts = defaultdict(int)
            for event in self.attendance_events:
                employee_counts[event['employee_name']] += 1
            
            for name, count in sorted(employee_counts.items()):
                avg_confidence = np.mean([e['confidence'] for e in self.attendance_events 
                                        if e['employee_name'] == name])
                print(f"├─ {name}: {count} recognition(s) (avg conf: {avg_confidence:.3f})")
        
        # Recommendations
        print(f"\n💡 RECOMMENDATIONS:")
        if recognition_rate < 50:
            print("├─ Low recognition rate - check employee registration quality")
        if total_errors > self.stats['processed_frames'] * 0.1:
            print("├─ High error rate - check system stability")
        if processing_rate < 5:
            print("├─ Low processing rate - consider reducing frame skip ratio")
        if avg_time > 200:
            print("├─ Slow AI processing - consider lighter model or GPU acceleration")
        
        print("└─ Session analysis completed")
    
    def stop_processing(self):
        """Stop real-time processing"""
        self.processing_active = False
        print("⏹️ Processing stop requested")
    
    def export_session_data(self, include_debug_info=True):
        """Export session data with enhanced information"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        try:
            # Ensure exports directory exists
            exports_dir.mkdir(exist_ok=True)
            
            # Export attendance events
            if self.attendance_events:
                events_data = []
                for event in self.attendance_events:
                    event_copy = event.copy()
                    event_copy['timestamp'] = datetime.fromtimestamp(event['timestamp']).isoformat()
                    event_copy['bbox'] = [float(x) for x in event['bbox']]  # Ensure JSON serializable
                    events_data.append(event_copy)
                
                events_path = exports_dir / f'attendance_events_{timestamp}.json'
                with open(events_path, 'w') as f:
                    json.dump(events_data, f, indent=2, default=str)
                print(f"✅ Attendance events exported: {events_path}")
                
                # Also export as CSV for easy analysis
                import pandas as pd
                df_events = pd.DataFrame(events_data)
                csv_path = exports_dir / f'attendance_events_{timestamp}.csv'
                df_events.to_csv(csv_path, index=False)
                print(f"✅ CSV export: {csv_path}")
            
            # Export comprehensive session statistics
            session_stats = {
                'session_info': {
                    'start_time': datetime.fromtimestamp(self.stats['session_start']).isoformat(),
                    'duration_minutes': (time.time() - self.stats['session_start']) / 60,
                    'total_frames': self.stats['total_frames'],
                    'processed_frames': self.stats['processed_frames'],
                    'faces_detected': self.stats['faces_detected'],
                    'employees_recognized': self.stats['employees_recognized']
                },
                'performance_metrics': {
                    'avg_processing_time_ms': float(np.mean(self.stats['processing_times'])) if self.stats['processing_times'] else 0,
                    'min_processing_time_ms': float(np.min(self.stats['processing_times'])) if self.stats['processing_times'] else 0,
                    'max_processing_time_ms': float(np.max(self.stats['processing_times'])) if self.stats['processing_times'] else 0,
                    'processing_times_history': list(self.stats['processing_times'])
                },
                'error_analysis': self.stats['errors'],
                'camera_stats': self.camera_manager.get_camera_stats(),
                'system_info': {
                    'ai_system_mock_mode': getattr(self.ai_system, 'mock_mode', False),
                    'cooldown_period_seconds': self.cooldown_period,
                    'frame_skip_ratio': self.process_every_n_frames
                }
            }
            
            if include_debug_info:
                session_stats['debug_info'] = {
                    'recognition_history': [
                        {
                            'timestamp': datetime.fromtimestamp(r['timestamp']).isoformat(),
                            'employee_name': r['employee_name'],
                            'confidence': r['confidence']
                        }
                        for r in self.recognition_history
                    ],
                    'active_cooldowns': {
                        str(emp_id): datetime.fromtimestamp(last_time).isoformat()
                        for emp_id, last_time in self.employee_cooldowns.items()
                    }
                }
            
            stats_path = exports_dir / f'session_stats_{timestamp}.json'
            with open(stats_path, 'w') as f:
                json.dump(session_stats, f, indent=2, default=str)
            
            print(f"✅ Session statistics exported: {stats_path}")
            
            return {
                'events_file': events_path if self.attendance_events else None,
                'stats_file': stats_path,
                'csv_file': csv_path if self.attendance_events else None
            }
            
        except Exception as e:
            print(f"❌ Export failed: {e}")
            return None
    
    def create_attendance_report(self):
        """Create a formatted attendance report"""
        if not self.attendance_events:
            print("⚠️ No attendance events to report")
            return
        
        print(f"\n📋 ATTENDANCE REPORT")
        print("=" * 50)
        print(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Session duration: {(time.time() - self.stats['session_start'])/60:.1f} minutes")
        print("-" * 50)
        
        # Group events by employee
        employee_events = defaultdict(list)
        for event in self.attendance_events:
            employee_events[event['employee_name']].append(event)
        
        # Display each employee's activity
        for employee_name, events in sorted(employee_events.items()):
            print(f"\n👤 {employee_name}:")
            print(f"├─ Total recognitions: {len(events)}")
            
            avg_confidence = np.mean([e['confidence'] for e in events])
            print(f"├─ Average confidence: {avg_confidence:.3f}")
            
            # Show chronological events
            events_sorted = sorted(events, key=lambda x: x['timestamp'])
            print(f"└─ Timeline:")
            
            for i, event in enumerate(events_sorted):
                timestamp_str = datetime.fromtimestamp(event['timestamp']).strftime('%H:%M:%S')
                marker = "├─" if i < len(events_sorted) - 1 else "└─"
                print(f"   {marker} {timestamp_str} - Confidence: {event['confidence']:.3f}")

# Initialize attendance processor with comprehensive checks
print("\n🔄 INITIALIZING ATTENDANCE PROCESSOR:")

try:
    # Check if required components are available
    if not all([ai_system, db, camera_manager]):
        missing_components = []
        if not ai_system:
            missing_components.append("AI System")
        if not db:
            missing_components.append("Database")
        if not camera_manager:
            missing_components.append("Camera Manager")
        
        print(f"❌ Missing components: {', '.join(missing_components)}")
        print("Please run previous cells successfully first")
        processor = None
    else:
        processor = AttendanceProcessor(ai_system, db, camera_manager)
        if processor.available:
            print("✅ Attendance Processor initialized successfully")
        else:
            print("❌ Attendance Processor initialization failed")
            processor = None

except Exception as e:
    print(f"❌ Processor initialization error: {e}")
    processor = None

# Display usage instructions
print("\n🎯 REAL-TIME PROCESSING OPTIONS:")
if processor and processor.available:
    print("📊 Main Functions:")
    print("├─ processor.start_real_time_processing(duration_minutes=5)")
    print("├─ processor.start_real_time_processing(duration_minutes=10, show_annotated_frames=True)")
    print("├─ processor.stop_processing()")
    print("└─ processor.export_session_data()")
    
    print("\n📋 Analysis Functions:")
    print("├─ processor.create_attendance_report()")
    print("├─ processor.export_session_data(include_debug_info=True)")
    print("└─ Check exports/ folder for saved data")
    
    print("\n⚙️ Advanced Options:")
    print("├─ processor.cooldown_period = 10  # Reduce cooldown to 10 seconds")
    print("├─ processor.process_every_n_frames = 5  # Process every 5th frame")
    print("└─ processor.display_update_interval = 1.0  # Update display every second")
else:
    print("❌ Processor not available")

print("\n💡 QUICK START GUIDE:")
print("1. Ensure employees are registered (Cell 3)")
print("2. Ensure camera is detected and working")
print("3. Run: processor.start_real_time_processing(duration_minutes=2)")
print("4. Watch for real-time recognition results")
print("5. Export data when finished")

print("\n⚠️ TROUBLESHOOTING:")
print("├─ Camera issues: Run camera_manager.detect_cameras()")
print("├─ No recognitions: Check employee registration quality")
print("├─ High errors: Check system resources and connections")
print("└─ Slow processing: Increase process_every_n_frames value")

print("=" * 35)

🎥 REAL-TIME CAMERA PROCESSING

🔄 INITIALIZING ATTENDANCE PROCESSOR:
✅ Attendance Processor initialized
✅ Attendance Processor initialized successfully

🎯 REAL-TIME PROCESSING OPTIONS:
📊 Main Functions:
├─ processor.start_real_time_processing(duration_minutes=5)
├─ processor.start_real_time_processing(duration_minutes=10, show_annotated_frames=True)
├─ processor.stop_processing()
└─ processor.export_session_data()

📋 Analysis Functions:
├─ processor.create_attendance_report()
├─ processor.export_session_data(include_debug_info=True)
└─ Check exports/ folder for saved data

⚙️ Advanced Options:
├─ processor.cooldown_period = 10  # Reduce cooldown to 10 seconds
├─ processor.process_every_n_frames = 5  # Process every 5th frame
└─ processor.display_update_interval = 1.0  # Update display every second

💡 QUICK START GUIDE:
1. Ensure employees are registered (Cell 3)
2. Ensure camera is detected and working
3. Run: processor.start_real_time_processing(duration_minutes=2)
4. Watch for real-ti